# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [102]:
import pandas as pd
import numpy as np
# импортируем алгоритмы машинного обучения из библиотеки sklearn
from sklearn.tree import DecisionTreeClassifier # решающее дерево
from sklearn.ensemble import RandomForestClassifier #  случайные деревья
from sklearn.linear_model import LogisticRegression  #   логистическая регрессия

from sklearn.metrics import accuracy_score   # метрика подсчета "точности"
from sklearn.model_selection import train_test_split # метод разделения на выборки

import time

In [103]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [105]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [106]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Проверим фитчи на мультиколлениарность, которая может стать причиной переобучаемости модели.
Для этого построим таблицу корреляции между фитчами.

In [107]:
df.corr().round(2)

,calls,minutes,messages,mb_used,is_ultra
calls,1.00,0.98,0.18,0.29,0.21
minutes,0.98,1.00,0.17,0.28,0.21
messages,0.18,0.17,1.00,0.20,0.20
mb_used,0.29,0.28,0.20,1.00,0.20
is_ultra,0.21,0.21,0.20,0.20,1.00


между фитчами Calls и  minutes определенно есть линейная зависимость. Думаю логичным будет из обучающей выборки убрать стольбец с колличеством звонков. так как в тарифных планах используется информация по количеству минут

## Разбейте данные на выборки

Разобъём наш датасет на обучающую, валидационную и тестовую выборки.
На тренировочном фрейме наша модель будет обучаться. На валидационной мы будем смотреть как влияют гиперпараметры на точность модели. И в финале мы проверим выбранные нами наилучшие настройки модели на тестовом фрейме. 

Отберем обучающие и целевые переменные у выборок

In [108]:
features = df.drop(['calls','is_ultra'], axis=1) # извлечем признаки 
target = df['is_ultra'] # извлечем целевой признак

In [109]:
features_learn, features_test, target_learn, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.25, 
                                                                            random_state=12345,
                                                                            stratify=target)

print(f'тренировочная: {features_learn.shape}, тестовая: {features_test.shape}')

тренировочная: (2410, 3), тестовая: (804, 3)


У тренировочной выборки отделим 25 процентов для проверки и сравнения точности моделей перед использованием на тестовой выборке

In [110]:
features_train, features_valid, target_train, target_valid = train_test_split(features_learn, 
                                                                              target_learn , 
                                                                              test_size=0.25, 
                                                                              random_state=12345,
                                                                              stratify=target_learn)

print(f'обучающая: {features_train.shape}, проверочная: {features_valid.shape}')

обучающая: (1807, 3), проверочная: (603, 3)


In [111]:
# код ревьюера для проверки
target_train.mean(), target_valid.mean(), target_test.mean()

(0.30658550083010516, 0.3067993366500829, 0.30597014925373134)

## Исследуйте модели

### посмотрим как отработает алгоритм обучения - решающие деревья 

In [134]:
scores_data = pd.DataFrame()

In [136]:
for depth in range(1,11):
    for split in range(2,42,5):
        for leaf in range(1,5):
            model = DecisionTreeClassifier(max_depth = depth,
                                           random_state=12345, 
                                           criterion='gini',
                                           min_samples_split = split,
                                           min_samples_leaf = leaf)
            model.fit(features_train, target_train)
            train_score = model.score(features_train, target_train)
            valid_score = model.score(features_valid, target_valid)
            test_score =  model.score(features_test, target_test)
            
            temp_scores_data = pd.DataFrame({'max_depth':[depth],
                                             'min_samples_split':[split],
                                             'min_samples_leaf':[leaf],
                                             'train_score':[train_score],
                                             'valid_score':[valid_score],
                                             'test_score':[test_score]})
            scores_data = scores_data.append(temp_scores_data)
            

In [138]:
scores_data

,max_depth,min_samples_split,min_samples_leaf,train_score,valid_score,test_score
0,1,2,1,0.753735,0.749585,0.752488
0,1,2,2,0.753735,0.749585,0.752488
0,1,2,3,0.753735,0.749585,0.752488
0,1,2,4,0.753735,0.749585,0.752488
0,1,7,1,0.753735,0.749585,0.752488
...,...,...,...,...,...,...
0,10,32,4,0.840620,0.792703,0.796020
0,10,37,1,0.841727,0.802653,0.808458
0,10,37,2,0.840620,0.805970,0.810945
0,10,37,3,0.838406,0.805970,0.812189


In [140]:
sores_data_long = pd.melt(scores_data, id_vars = ['max_depth','min_samples_split','min_samples_leaf'], 
                          value_vars = ['train_score','valid_score','test_score'], 
                          var_name = 'set_type', value_name = 'score')
scores_data_long

,max_depth,min_samples_split,min_samples_leaf,set_type,score
0,1,2,1,train_score,0.753735
1,1,2,2,train_score,0.753735
2,1,2,3,train_score,0.753735
3,1,2,4,train_score,0.753735
4,1,7,1,train_score,0.753735
...,...,...,...,...,...
955,10,32,4,test_score,0.796020
956,10,37,1,test_score,0.808458
957,10,37,2,test_score,0.810945
958,10,37,3,test_score,0.812189


In [ ]:
import seaborn as sns
sns.lineplot(x=['max_depth','min_samples_split','min_samples_leaf'],
             y='score', 
             hue='set_type', 
             data=scores_data_long)

In [112]:
%%time
best_model = None
best_result = 0
best_depth = 0
best_split = 0
best_leaf = 0

for depth in range(1,11):
    for split in range(2,42,5):
        for leaf in range(1,5):
            model = DecisionTreeClassifier(max_depth = depth,
                                           random_state=12345, 
                                           criterion='gini',
                                           min_samples_split = split,
                                           min_samples_leaf = leaf) #, min_samples_split=,min_samples_leaf=)
            model.fit(features_train, target_train)
            predictions_valid = model.predict(features_valid)
            result = accuracy_score(target_valid, predictions_valid).round(3) # вычислим 
            # print(f'глубина дерева = {depth}, split={split}, leaf={leaf}, точность: {result}')
            if result > best_result:
                best_depth = depth # сохраним наилучшую модель
                best_result = result # сохраним наилучшее значение метрики accuracy на валидационных данных
                best_model = model
                best_split = split
                best_leaf = leaf
print('--------------------------------------')     
print(f'Наилучшая модель алгоритма "решающее дерево" при параметрах:\
\nглубина дерева = {best_depth},\
      \nМинимальное количество выборок в ветке: {best_split},\
      \nМинимальное количество образцов в листе: {best_leaf},\
      \nточность: {best_result}\n') 

--------------------------------------
Наилучшая модель алгоритма "решающее дерево" при параметрах:
глубина дерева = 8,      
Минимальное количество выборок в ветке: 22,      
Минимальное количество образцов в листе: 2,      
точность: 0.811

CPU times: user 1.49 s, sys: 0 ns, total: 1.49 s
Wall time: 1.49 s


In [113]:
# код ревьюера для проверки - сохраним модель решающего дерева, чтобы оценить её переобученность
best_dt_model = best_model
best_dt_model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=22,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

### посмотрим как отработает алгоритм обучения - случайный лес

In [114]:
%%time
best_est = 0
best_model = None
best_result = 0
best_depth = 0

for est in range(10,110,10): # переберём гиперпараметр - количество деревьев
    for depth in range(10,51,10):  # переберём гиперпараметр - максимальная глубина деревьев
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid,predictions_valid).round(4)
        if result > best_result:
            best_est = est # сохраним наилучшие параметры
            best_depth = depth
            best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных
            best_model = model
print('--------------------------------------')     
print(f'Наилучшая модель алгоритма "Случайный лес" при параметрах:\
\nколичество дереьев в лесу: {best_est},\
\nмаксимальная глубина: {best_depth},\
\nточность: {best_result}\n')    

--------------------------------------
Наилучшая модель алгоритма "Случайный лес" при параметрах:
количество дереьев в лесу: 30,
максимальная глубина: 10,
точность: 0.8143

CPU times: user 7.6 s, sys: 55.5 ms, total: 7.65 s
Wall time: 7.76 s


In [115]:
# код ревьюера для проверки
best_est

30

In [116]:
# код ревьюера для проверки - сохраним модель случайного леса
best_rf_model = best_model
best_rf_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

### посмотрим как отработает алгоритм обучения - логистическая регрессия

In [117]:
%%time
lr_model = LogisticRegression(random_state=12345)
lr_model.fit(features_train, target_train)
predictions_valid = lr_model.predict(features_valid)
result = accuracy_score(target_valid,predictions_valid).round(3)
print(f'Точность алгоритма логистичекая регрессия: {result}\n')

Точность алгоритма логистичекая регрессия: 0.718

CPU times: user 9.47 ms, sys: 42 µs, total: 9.51 ms
Wall time: 7.96 ms


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


У этого алгоритма минимальное количество гиперпараметров - поэтому оно лучше избегает переобучения и быстро отрабатывает, но он показал наихудшую точность прогноза на валидационной выборке.

In [118]:
# код ревьюера для объяснения
best_model = None
best_result = 0
for penalty in ['l1', 'l2']:
    for C in [0.001, 0.01, 1.0, 5.0, 10.0, 20.0]:
        lr_model = LogisticRegression(random_state=12345, penalty=penalty, C=C, solver='liblinear') 
        lr_model.fit(features_train, target_train) 
        result = accuracy_score(target_valid, model.predict(features_valid))
        if result > best_result:
            best_lr_model = lr_model
            best_result = result
print("Accuracy лучшей модели логистической регрессии на валидационной выборке:", best_result)
#best_lr_model = best_model
print('Лучшая модель логистической регрессии:', best_lr_model, sep='\n')

Accuracy лучшей модели логистической регрессии на валидационной выборке: 0.8009950248756219
Лучшая модель логистической регрессии:
LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)


In [119]:
# код ревьюера для проверки - сохраним модель логистической регрессии
# best_lr_model = best_model
# best_lr_model

## Проверьте модель на тестовой выборке

Лучшая модель с максимальной долей верных ответов (accuracy) получилась на алгоритме RandomForest с гиперпараметром n_estimators = 10, max_depth = 10
отрабатывает долго но выборки сейчас не большие по этому воспользуемся ей на тестовой выборке

In [121]:
best_rf_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [122]:
# model = RandomForestClassifier(n_estimators=10, max_depth=40, random_state=12345)
# model.fit(features_train, target_train)
predictions_test = best_rf_model.predict(features_test)
result = accuracy_score(target_test,predictions_test).round(4)
print(f'точность: {result}')

точность: 0.8184


## (бонус) Проверьте модели на адекватность

Для проверки на адекватность сначала посмотрю в чем расхождение предсказаний от реальных данных 
Соеденю в один датафрейм нужные данные..

In [123]:
df_test = pd.concat([features_test, target_test],axis=1)

In [124]:
df_test.loc[:,'predict'] = list(predictions_test)

In [125]:
df_test['is_ultra'].value_counts()
#df_test['is_ultra'].value_counts(normalize=True)

0    558
1    246
Name: is_ultra, dtype: int64

In [126]:
df_test['predict'].value_counts(normalize=True)

0    0.800995
1    0.199005
Name: predict, dtype: float64

В требуемой классификации всего две категории с соотношением друг к другу 70(не ультра) и 30(ультра) процентов в реальных данных.
В предсказанных соотношение категорий уже 80 и 20 процентов, а точность прогноза составила 80 процентов совпадений. 

Проведу эксперимент - сгенерирую случайным образом массив с частотой категорий как у реальных данных (70 на 30), и посмотрим что на это скажет метод проверки точности. 

In [127]:
result = accuracy_score(target_test,np.random.binomial(1,0.3,804)).round(4)
print(f'точность: {result}')

точность: 0.597


Сделаю это раз 100 , чтоб понять - возможно ли случайным образом получить результат 80 процентов совпадений .

In [128]:
list_random_accuracy = []      # создам пустой список - куда буду складывать получаемый accuracy
for i in range(1,101):   #  инициализирую цикл  для получения 100 рандомных последовательности
    result = accuracy_score(target_test,np.random.binomial(1,0.3,804)).round(4)   # посчитаем методом accuracy из библиотеки sklearn
    list_random_accuracy.append(result)         #
pd.Series(list_random_accuracy).describe()      # преобразуем в серию для удобного получения описательных характеристик

count    100.000000
mean       0.576667
std        0.017505
min        0.527400
25%        0.565900
50%        0.578400
75%        0.587400
max        0.624400
dtype: float64

Полученная последовательность только старается быть с пропорциями 0 и 1 как в реальном датафрейме - сделаем пропорции точно такими же. 

In [129]:
# другой способ получить рандомную последовательность 0 и 1 но уже с точным количеством как в реальном датафрейме
pd.Series(np.random.choice(([0]*558 + [1]* 246),804,replace=False)).value_counts()

0    558
1    246
dtype: int64

In [130]:
list_random_accuracy = []      # создам пустой список - куда буду складывать получаемый accuracy
for i in range(1,101):   #  инициализирую цикл  для получения 100 рандомных последовательности
    
    result = accuracy_score(target_test,np.random.choice(([0]*563 + [1]* 241),804,replace=False)).round(4)   #
    list_random_accuracy.append(result)         # добавим результат в список
pd.Series(list_random_accuracy).describe()     # преобразуем в серию для удобного получения описательных характеристик

count    100.000000
mean       0.579845
std        0.015481
min        0.538600
25%        0.568400
50%        0.583300
75%        0.590800
max        0.615700
dtype: float64

Зря я затеил эти эксперименты
Получается что если бы я просто заполнил тестовый массив самой частотной категорией - то есть 0(не ультра) , то совпадения составят 70 процентов.  
Логично... в рандомных экспериментах (против частотного) ошибки приросли на обоих категориях.


Ну что ж. 
При подстановке самого частотного случая  и при 100 экспериментах получения случайной последовательности(с соблюдением пропорций) видно что случайно получить результат точности прогноза модели мало вероятно. Получается что модель адекватна.(на 10 процентов адекватней "частотного")

In [131]:
# код ревьюера для объяснения

from sklearn.dummy import DummyClassifier

dummy_model = DummyClassifier(strategy= 'most_frequent').fit(features_train, target_train)
dummy_pred = dummy_model.predict(features_test)
print("Accuracy базовой модели на тестовой выборке:", accuracy_score(target_test, dummy_pred))

Accuracy базовой модели на тестовой выборке: 0.6940298507462687


In [132]:
# код ревьюера для проверки

models = {
    'Базовая модель': dummy_model,
    'Логистическая регрессия без подбора': lr_model,
    'Логистическая регрессия': best_lr_model,
    'Дерево решений': best_dt_model,
    'Случайный лес': best_rf_model
}

for model_name, model in models.items():
    print(f'{model_name}:')
    print("\tAccuracy на валидационной выборке:", accuracy_score(target_valid, model.predict(features_valid)))
    print("\tAccuracy на тестовой выборке:     ", accuracy_score(target_test, model.predict(features_test)))
    print()

Базовая модель:
	Accuracy на валидационной выборке: 0.693200663349917
	Accuracy на тестовой выборке:      0.6940298507462687

Логистическая регрессия без подбора:
	Accuracy на валидационной выборке: 0.7180762852404643
	Accuracy на тестовой выборке:      0.7164179104477612

Логистическая регрессия:
	Accuracy на валидационной выборке: 0.7064676616915423
	Accuracy на тестовой выборке:      0.7002487562189055

Дерево решений:
	Accuracy на валидационной выборке: 0.8109452736318408
	Accuracy на тестовой выборке:      0.8034825870646766

Случайный лес:
	Accuracy на валидационной выборке: 0.814262023217247
	Accuracy на тестовой выборке:      0.818407960199005

